In [ ]:
import gcsfs
import xarray as xr
import zarr
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import rasterio as rio
from rasterio.features import rasterize
from skimage.measure import block_reduce
import time
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed
from satextractor.utils import tqdm_joblib
import pandas as pd
import geopandas as gpd
from oxeo.water.models.utils import merge_masks
from oxeo.water.metrics import metrics
import gcsfs
fs = gcsfs.GCSFileSystem(project='oxeo-main')

In [ ]:
import seaborn as sns

In [ ]:

shapefile = gpd.read_file("../../ReservoirWatchHack/Data/Contextual_Shapefiles/reservoirs.shp")



In [ ]:
shapefile.geometry.to_file('india_reservoirs.geojson', driver='GeoJSON')  

In [ ]:
shapefile

In [ ]:
patches_paths = fs.ls('oxeo-water/eo')
patches_paths



In [ ]:

ls "../../ReservoirWatchHack/Data/CWC_WaterLevels/"

In [ ]:
dam = "Harangi"

patches_paths = fs.ls('oxeo-water/eo/harangi')
print(patches_paths)

print("Merging masks")
full_mask,dates= merge_masks(patches_paths, 1000,datetime(2011,2,9),datetime(2021,10,25))
full_mask = full_mask.astype(np.uint8)
print("Masks merged")
dates_str = [d.strftime("%Y-%m-%d") for d in dates]
dam_df = pd.read_csv(f"../../ReservoirWatchHack/Data/CWC_WaterLevels/{dam}.csv")
dam_df = dam_df[dam_df.FLOW_DATE.isin(dates_str)].sort_values(by="FLOW_DATE")
dates_mask = pd.Series(dates_str).isin(dam_df.FLOW_DATE).values

full_mask = full_mask[np.where(dates_mask)[0], ...]

In [ ]:
full_mask.shape

In [ ]:
print("Getting areas")
areas = metrics.segmentation_area(full_mask)
data = pd.DataFrame({"res_level": dam_df.RES_LEVEL_FT.astype(float), "areas": areas})
from scipy import stats
val, p = stats.pearsonr(data['res_level'], data['areas'])
print("Plotting")
sns.lmplot(x="res_level", y="areas", data=data)
plt.legend(labels=[f'{dam} pearson {val:.2f} with p-value {p:.5f}'])
plt.show()